In [1]:
import torch
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

import llmtoolkit
from llmtoolkit import (
    get_args,
    get_accelerate_model,
    build_data_module,
    get_unique_key,
    train,
    TrainingArguments,
    ModelArguments,
)

/Users/ltzhang/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[llm toolkit]: Detected 0 GPU(s)
[llm toolkit]: Detected 0 XPU(s)


/Users/ltzhang/anaconda3/envs/py310/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/ltzhang/anaconda3/envs/py310/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <EA7F9DF5-8854-31D8-89D4-BD566CAF4DEA> /Users/ltzhang/anaconda3/envs/py310/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <ECC148AF-20FF-3EEE-BC75-4DD3E7455393> /Users/ltzhang/anaconda3/envs/py310/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/ltzhang/anaconda3/envs/py310/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbyte

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
MODEL = "/Users/ltzhang/repo/llm-toolkit/models/Llama-2-1.3b-hf"
model_args = ModelArguments(model_name_or_path=MODEL)
training_args = TrainingArguments(max_steps=5, group_by_length = False, optim = "adamw_torch")


In [3]:
model, tokenizer = get_accelerate_model(model_args, training_args)
print(model)

[llm toolkit]: loading base model /Users/ltzhang/repo/llm-toolkit/models/Llama-2-1.3b-hf...
[llm toolkit]: LOADING UNQUANTIZED MODEL


/Users/ltzhang/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.42s/it]

[llm toolkit]: adding special tokens, {'pad_token': '<unk>'}
[llm toolkit]: pad_token: <unk>
[llm toolkit]: eos_token: </s>
[llm toolkit]: bos_token: <s>
[llm toolkit]: unk_token: <unk>
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=

In [4]:
key_list = [key for key, _ in model.named_modules()]

def _get_submodules(model, key):
    parent = model.get_submodule(".".join(key.split(".")[:-1]))
    target_name = key.split(".")[-1]
    target = model.get_submodule(key)
    return parent, target, target_name

In [5]:
import bitsandbytes as bnb
from bitsandbytes.nn import LinearNF4

from transformers.pytorch_utils import Conv1D

In [6]:
parent, child, child_name = _get_submodules(model, 'model.layers.0.self_attn.q_proj')

in_features = child.in_features
out_features = child.out_features

bnb_4bit_compute_dtype = torch.bfloat16
bnb_4bit_use_double_quant = False
bnb_4bit_quant_type = 'nf4'
bnb_4bit_quant_storage = torch.bfloat16

new_module = bnb.nn.Linear4bit(
    in_features,
    out_features,
    child.bias is not None,
    bnb_4bit_compute_dtype,
    compress_statistics=bnb_4bit_use_double_quant,
    quant_type=bnb_4bit_quant_type,
    # quant_storage=bnb_4bit_quant_storage,
)
new_value_weight = bnb.nn.Params4bit(child._parameters["weight"], requires_grad=True)
new_module.weight = new_value_weight
if child.bias is not None:
    new_value_bias = bnb.nn.Params4bit(child._parameters["bias"], requires_grad=True)
    new_module.bias = new_value_bias
    
if getattr(child, "state", None) is not None:
    new_module.state = child.state


In [7]:
new_value_weight.requires_grad

True

In [58]:
for n,m in new_module.named_modules():
    print(m)

Linear4bit(in_features=2048, out_features=2048, bias=False)


In [7]:
def replace_module(parent, child_name, new_module, child):
    setattr(parent, child_name, new_module)
    new_module.to(child.weight.device)
    meta = torch.device("meta")
    for name, module in new_module.named_modules():
        if not any(p.device == meta for p in module.parameters()):
            module.to(child.weight.device)
    pass

In [8]:
replace_module(parent, child_name, new_module, child)

In [9]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
      (1-

In [62]:
for n, p in model.named_parameters():
    if p.requires_grad is False:
        print(n)

In [ ]:
def _replace_module(parent, child_name, new_module, child):
    setattr(parent, child_name, new_module)
    # It's not necessary to set requires_grad here, as that is handled by
    # _mark_only_adapters_as_trainable

    # child layer wraps the original module, unpack it
    if hasattr(child, "base_layer"):
        child = child.base_layer

    if not hasattr(new_module, "base_layer"):
        if hasattr(new_module, "W_q"):  # HQQ
            new_module.W_q = child.W_q
        else:
            new_module.weight = child.weight
        if hasattr(child, "bias"):
            new_module.bias = child.bias

    if getattr(child, "state", None) is not None:
        if hasattr(new_module, "base_layer"):
            new_module.base_layer.state = child.state
        else:
            new_module.state = child.state
        new_module.to(child.weight.device)

    meta = torch.device("meta")
    # dispatch to correct device
    for name, module in new_module.named_modules():
        if ("lora_"in name) or ("ranknum" in name):
            weight = (
                child.qweight
                if hasattr(child, "qweight")
                else child.W_q
                if hasattr(child, "W_q")
                else child.weight
                if hasattr(child, "weight")
                else next(child.parameters())
            )
            if not any(p.device == meta for p in module.parameters()):
                module.to(weight.device)

In [10]:
data_module = build_data_module(tokenizer, "gsm8k")

[llm toolkit]: Sample 4500 of the training set:
{'question': 'Uncle Bradley has a $1000 bill that he wants to change into smaller bills so he could give them to his nieces and nephews. He wants to change 3/10 of the money into $50 bills while the rest into $100 bills. How many pieces of bills will Uncle Bradley have in all?', 'answer': 'There will be $1000 x 3/10 = $<<1000*3/10=300>>300 worth of money that will be changed into $50 bills.\nUncle Bradley will have $300/$50 = <<300/50=6>>6 pieces of $50 bills.\nHe will change $1000 - $300 = $<<1000-300=700>>700 worth of money into $100 bills.\nSo there will be $700/$100 = <<700/100=7>>7 pieces of $100 bills.\nThus, there will be a total of 6 + 7 = <<6+7=13>>13 pieces of $50 and $100 bills in all.\n#### 13', 'input': 'Question: Uncle Bradley has a $1000 bill that he wants to change into smaller bills so he could give them to his nieces and nephews. He wants to change 3/10 of the money into $50 bills while the rest into $100 bills. How many

In [68]:
training_args

TrainingArguments(output_dir='default_output', overwrite_output_dir=False, do_train=True, do_eval=False, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=0.0002, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=0.3, num_train_epochs=3.0, max_steps=5, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.03, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='default_output/runs/Nov19_20-50-40_Aarons-MacBook-Pro.local', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=10, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=250, save_total_l

In [11]:
train(model, tokenizer, data_module["train_dataset"], data_module["eval_dataset"], data_module["data_collator"], training_args, "replace")


[llm toolkit]: trainable params: 1345423360 || all params: 1345423360 || trainable%: 100.00


/Users/ltzhang/anaconda3/envs/py310/lib/python3.10/site-packages/llmtoolkit/trainer.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer_llmtoolkit.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
max_steps is given, it will override any value given in num_train_epochs


[llm toolkit]: device map: {'': 'mps'}
[llm toolkit]: *** Train ***


  0%|          | 0/5 [00:00<?, ?it/s]

FP4 quantization state not initialized. Please call .cuda() or .to(device) on the LinearFP4 layer first.


AssertionError: 